In [ ]:
%config Completer.use_jedi = False

In [ ]:
%%capture
!cp -r /kaggle/input/label-assignment-nfl/label_assignment/gmmreg-install /kaggle/working/gmmreg-install
%cd /kaggle/working/gmmreg-install/src
!python setup.py install --user
%cd /kaggle/working
%rm -r /kaggle/working/gmmreg-install

In [ ]:
%%capture
!cp -r /kaggle/input/nfl-yolov5-models-v2/yolov5 /kaggle/working/yolov5

In [ ]:
import os
import sys

from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
is_sub = len(os.listdir('/kaggle/input/nfl-health-and-safety-helmet-assignment/test')) != 6

# Inference

In [ ]:
# N_VIDEOS = 6
deepsort_cfg = {
    'model_path': '/kaggle/input/deepsort-nfl-models/deepsort_nfl_v2.pth',
    'max_dist': 0.05,
    'min_confidence': 0.35,
    'max_iou_distance': 0.6,
    'max_age': 30,
    'n_init': 1,
    'nn_budget': 80,
}
labeler_cfg_Endzone = {
    'level': 3, 
    'scales':  [1, 0.2, 0.1], 
    'lambdas': [0.1, 0.04, 0.02], 
    'iters':   [30, 20, 10],
    'n_grid': 5
}
labeler_cfg_Sideline = {
    'level': 3, 
    'scales':  [1, 0.5, 0.25], 
    'lambdas': [1, 0.02, 0.25], 
    'iters':   [30, 20, 10],
    'n_grid': 5
}
videos = os.listdir('/kaggle/input/nfl-health-and-safety-helmet-assignment/test')#[:N_VIDEOS]
videos = list(map(lambda x: x.split('.')[0], videos))
videos.sort()
base_video_dir = '/kaggle/input/nfl-health-and-safety-helmet-assignment/test'

## Detection with YoloV5

In [ ]:
YOLO_WEIGHTS = [
    '/kaggle/input/nfl-yolov5-models-v2/artifacts/run_183sllab_model:v0/best.pt',
    '/kaggle/input/nfl-yolov5-models-v2/artifacts/run_3fc0fdzr_model:v0/best.pt',
    '/kaggle/input/nfl-yolov5-models-v2/artifacts/run_1nqxdcxw_model:v0/best.pt', # yolov5x6_FOLD01 (validated on folds 0 and 1)
    '/kaggle/input/nfl-yolov5-models-v2/artifacts/run_m24omzoc_model:v0/best.pt', # yolov5x6_FOLD40 (validated on folds 2 and 3)
    '/kaggle/input/nfl-yolov5-models-v2/artifacts/run_3pena160_model:v0/best.pt', # yolov5x6_FOLD40 (validated on folds 0 and 4)
]

In [ ]:
%cd yolov5
import glob
from pathlib import Path
all_detections = []
for weights in YOLO_WEIGHTS:
    bboxes = []
    for video in tqdm(glob.glob("/kaggle/input/nfl-health-and-safety-helmet-assignment/test/*.mp4")):
        !python detect.py --weights $weights --img 1280 --conf 0.25 --save-txt --save-conf --source $video > /dev/null 2>&1
        for file in glob.glob("runs/detect/exp/labels/*.txt"):
            bbox = pd.read_csv(file, sep = ' ', names = ['class', 'xc', 'yc', 'w', 'h', 'conf']).drop('class', axis = 1)
            bbox['video_frame'] = Path(file).stem
            bboxes.append(bbox)
        !rm -r runs/detect/exp
    bboxes = pd.concat(bboxes)
    bboxes['left'] = ((bboxes['xc'] - bboxes['w']/2) * 1280).astype(int)
    bboxes['top'] = ((bboxes['yc'] - bboxes['h']/2) * 720).astype(int)
    bboxes['width'] = (bboxes['w'] * 1280).astype(int)
    bboxes['height'] = (bboxes['h'] * 720).astype(int)
    all_detections.append(bboxes)
%cd ..

In [ ]:
# all_detections = [bboxes]

In [ ]:
# files = [
#     '/kaggle/input/nfl-csv-dataset/yolov5s6_run_2tasc22q_F0.csv',
#     '/kaggle/input/nfl-csv-dataset/yolov5s6_run_29r299qh_F1.csv',
#     '/kaggle/input/nfl-csv-dataset/yolov5s6_run_3skk47rd_F2.csv',
#     '/kaggle/input/nfl-csv-dataset/yolov5s6_run_20f824im_F4.csv'
# ]
# all_detections = []
# for i, file in enumerate(files):
#     det = pd.read_csv(file).rename(columns = {'w':'width', 'h':'height'})
#     det = det[det['video_frame'].apply(lambda x: '_'.join(x.split('_')[:-1])).isin(videos)].copy()                                
#     all_detections.append(det)

## Box Fusion

In [ ]:
sys.path.append("/kaggle/input/weighted-boxes-fusion-nfl")
from ensemble_boxes.ensemble_boxes_wbf import weighted_boxes_fusion
from nfl_wbf.utils import *

In [ ]:
all_detections = preprocess_detection_list(all_detections)
detections = fuse(all_detections, weighted_boxes_fusion, iou_thr=0.55) #change the name to fuse
detections = x1y1x2y2_to_ltwh(denormalize(detections))
detections.to_csv('detections.csv')

## Tracking with DeepSort

In [ ]:
sys.path.append('/kaggle/input/deepsort-nfl')
from deepsort.sort import sort_bboxes

In [ ]:
# sort_bboxes inputs bboxes for a single video 
# with the columns ['video_frame', 'left', 'width', 'top', 'height', 'conf']
# and the base dir for the videos (train/test) and hyper_params
# it outputs a dataframe with the columns
# ['video_frame', 'id', 'left', 'width', 'top', 'height']

## Expand height for better tracking
detections['height'] = detections['height']*2
detections['video'] = detections['video_frame'].apply(lambda x: '_'.join(x.split('_')[:-1]))
detections = detections.set_index('video')

tracked_detections = []
for video in tqdm(videos):
    bboxes_video = detections.loc[video].reset_index(drop = True)
    tracked_detections.append(sort_bboxes(bboxes_video, base_video_dir, **deepsort_cfg))
tracked_detections = pd.concat(tracked_detections).reset_index()
tracked_detections['height'] = tracked_detections['height']//2
tracked_detections.to_csv('tracked_detections.csv')

## Label Assignment

In [ ]:
sys.path.append("/kaggle/input/label-assignment-nfl")
from label_assignment.all import *
from label_assignment.helmet_assignment.score import NFLAssignmentScorer
from label_assignment.helmet_assignment.video import video_with_predictions

In [ ]:
# from gmmreg._core import run_multi_level
# from functools import partial
# import numpy as np
# class Register():
#     def __init__(self, algo = 'gmmreg', **kwargs):
#         if algo == 'gmmreg':
#             if 'n_grid' in kwargs:
#                 n_grid = kwargs.pop('n_grid')
#                 grid = np.linspace(-2, 2, n_grid)
#                 self.grid = np.array(np.meshgrid(grid, grid)).T.reshape(-1,2)
#             else:
#                 self.grid = None
#             self.algo = partial(run_multi_level, **kwargs)
#         else:
#             raise ValueError('Only gmmreg is implemented')
#     def __call__(self, src, trg):
#         if self.grid is None: grid = src
#         else: grid = self.grid
#         return self.algo(src, trg, grid)

In [ ]:
labeler_cfg_Endzone = {
    'level': 3, 
    'scales':  [1, 0.2, 0.1], 
    'lambdas': [0.1, 0.04, 0.02], 
    'iters':   [30, 20, 10],
    'n_grid': 5
}
# reg_gmm_Endzone = Register(**labeler_cfg_Endzone)
labeler_cfg_Sideline = {
    'level': 3, 
    'scales':  [1, 0.2, 0.1], 
    'lambdas': [0.1, 0.04, 0.02], 
    'iters':   [30, 20, 10],
    'n_grid': 5
}
# reg_gmm_Sideline = Register(**labeler_cfg_Sideline)

In [ ]:
# def match_videoV2(dl, theta, video):
#     video_dist = []
#     for frame in dl.frames:
#         xy_video, xy_tracking, labels_video, labels_tracking = dl(frame) 
#         xy_tracking = normalize(xy_tracking)
#         xy_video = normalize(xy_video)
#         xy_video = rotate(xy_video, theta)
#         if frame == 1:
#             _xy_video = xy_tracking
#         if 'Sideline' in video:
#             xy_video = reg_gmm_Sideline(xy_video, _xy_video)
#         else:
#             xy_video = reg_gmm_Endzone(xy_video, _xy_video)

#         dist = cdist(xy_video, xy_tracking)
#         M = linear_sum_assignment(dist)
#         _xy_video = xy_tracking[M[1]]
        
#         cost = 1/dist[M].mean()
#         dist = label_matrix(dist, labels_video, labels_tracking)
#         video_dist.append(dist * cost)
#     return video_dist 

In [ ]:
def track2sub(dl, video, **kwargs):
    theta = estimate_theta(dl, **kwargs)
    video_dist = match_videoV2(dl, theta, video)
    video_agg_dist = pd.concat(video_dist).groupby(level=0).agg('mean')
    video_labels, idx_video = assign_labels(dl, video_agg_dist)
    video_sub = build_submission_for_video(dl, video_labels, idx_video)
    return video_sub, theta

In [ ]:
# the input of DataLoader must be a dataframe of columns 
# video_frame, id, left, width, top, height
# it doesn't remove any boxes by default, to limit top 22, set 'top22 = True'
# Set is_train = True to load train track data and False to load test track data

dl = DataLoader(tracked_detections, is_train = False)

submission = []
for video in tqdm(dl.videos):

    ## Filtering the dataloader to return the data of a single video
    dl.filter_video(video = video)

    ## generating submission for a single video
    if 'Sideline' in video:
        video_sub, theta = track2subV2(dl, video, **labeler_cfg_Sideline)
    else:
        video_sub, theta = track2subV2(dl, video, **labeler_cfg_Endzone)
    submission.append(video_sub)

submission = pd.concat(submission)

## Fix sub

In [ ]:
sys.path.append("/kaggle/input/label-assignment-nfl")
from label_assignment.utils import fix_submission

In [ ]:
submission_fixed = fix_submission(submission)
submission_fixed.to_csv(index = False)
submission_fixed.to_csv('submission.csv', index = False)
submission_fixed

## Debug

In [ ]:
if not is_sub:
    gt_labels = pd.read_csv('/kaggle/input/nfl-health-and-safety-helmet-assignment/train_labels.csv')
    for video in tqdm(videos):
        scorer = NFLAssignmentScorer(gt_labels.query(f'video == "{video}.mp4"'))
        print(f'Score for Video {video:>21}: {scorer.score(submission_fixed):5.3f}')

In [ ]:
!rm -r /kaggle/working/yolov5